In [1]:
from glob import glob
import json
from tqdm import tqdm
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import json

In [2]:
data = []

with open('ada-002-facebook.jsonl') as fopen:
    for x in fopen:
        data.append(json.loads(x))

In [3]:
len(data)

192885

In [4]:
import math

math.comb(len(data),2)

18602215170

In [5]:
data[0]

{'v': [-0.033773023635149,
  -0.03641154244542122,
  -0.011886521242558956,
  -0.03778357058763504,
  -0.037546105682849884,
  0.02902369201183319,
  -0.0076780859380960464,
  0.018746666610240936,
  -0.013258550316095352,
  -0.035197824239730835,
  -0.010758555494248867,
  0.015554061159491539,
  -0.0008855524356476963,
  -0.019076481461524963,
  0.006744710262864828,
  -0.008555392734706402,
  0.025065915659070015,
  -0.005042866803705692,
  0.031398359686136246,
  -0.007137190084904432,
  0.004251311533153057,
  0.009716341271996498,
  0.012374646961688995,
  0.005098935216665268,
  -0.002503293566405773,
  -0.01372029073536396,
  0.0069393012672662735,
  -0.017941920086741447,
  -0.02440628781914711,
  0.009063308127224445,
  0.02474929392337799,
  0.012110795825719833,
  0.0031431340612471104,
  0.011193910613656044,
  -0.019010519608855247,
  -0.01094325166195631,
  0.015474905259907246,
  0.016042185947299004,
  -0.013667521066963673,
  0.010626628994941711,
  0.0116952285170555

In [6]:
vectors, texts = [], []

for d in data:
    vectors.append(d['v'])
    texts.append(d['cleaned_text'])

In [7]:
data[0]['cleaned_text']

'haii mat mlm.. Noraty Lesty... aku x pernah bergambar... nanti tanye boss aku kalau dia onzz... boleh'

In [8]:
texts[0]

'haii mat mlm.. Noraty Lesty... aku x pernah bergambar... nanti tanye boss aku kalau dia onzz... boleh'

In [9]:
from scipy.spatial import KDTree
import numpy as np

In [10]:
concat = np.array(vectors)
concat.shape

(192885, 1536)

In [11]:
%%time

kd_tree = KDTree(concat, leafsize = 40)

CPU times: user 2.55 s, sys: 24.1 ms, total: 2.57 s
Wall time: 2.57 s


In [12]:
concat[0]

array([-0.03377302, -0.03641154, -0.01188652, ..., -0.02255932,
        0.00241424, -0.03153028])

In [13]:
dist_list = []
ind_list = []
indices = []

def compute_neighbors(args):
    idx, x = args
    dist, ind = kd_tree.query(x, k=len(concat),workers=50)
    return idx, dist, ind

with ThreadPoolExecutor(max_workers=30) as executor:
    futures = list(tqdm(executor.map(compute_neighbors, enumerate(concat)), total=len(concat)))
    
    for future in tqdm(futures, total=len(concat)):
        if future:
            dist_list.append(future[1])
            ind_list.append(future[2])
            indices.append(future[0])

100%|██████████| 192885/192885 [00:00<00:00, 1150382.97it/s]


In [14]:
len(dist_list),len(ind_list),len(indices)

(192885, 192885, 192885)

In [16]:
indices

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [21]:
combined_data = list(zip(indices, dist_list, ind_list))

sorted_data = sorted(combined_data, key=lambda x: x[0])

sorted_indices_list, sorted_dist_list, sorted_index_list = zip(*sorted_data)

In [23]:
sorted_index_list[0]

array([     0, 149242, 190042, ...,  30543,  20483,   9985])

In [31]:
sorted_index_list[50][sorted_dist_list[50] <=0.45]

array([    50,  43644,  78217,  15770,  91392, 172623,  90454, 168396])

In [32]:
texts[50]

'Mohd Fuad Abdullah Sani kah'

In [33]:
texts[43644]

'Mohd Fuad Abdullah Sani takziah untuk kau sekeluarga'

In [31]:
dist_list[1][dist_list[1] <=0.45].shape

(17,)

In [ ]:
dist[dist > 0.8].shape

In [ ]:
import seaborn as sns

sns.distplot(dist[4:1000])

In [44]:
texts[50]

'Mohd Fuad Abdullah Sani kah'

In [45]:
texts[43644]

'Mohd Fuad Abdullah Sani takziah untuk kau sekeluarga'

In [49]:
index = [x for x in ind_list[50][dist_list[50] < 0.4]]
dist_index = [x for x in ind_list[50][dist_list[50] < 0.4]]

index

[24]

In [47]:
dist_index

[50, 43644]

In [48]:
len(dist_index),len(index)

(2, 2)

In [ ]:
[texts[x] for x in index]

In [ ]:
index = [x for x in ind[19999][dist[19999] <= 0.45]]
dist_index = [x for x in dist[19999][dist[19999] <= 0.45]]

index

In [ ]:
dist_index

In [ ]:
[texts[x] for x in index]

In [34]:
def construct_json(index):
    result = {
        'positive': [],
        'negative': []
    }
    
    result['positive'] = [x for x in sorted_index_list[index][sorted_dist_list[index] <= 0.45] if x != index]
    result['negative'] = [x for x in sorted_index_list[index][sorted_dist_list[index] > 0.8]]
    
    return result


result_json = {int(index): construct_json(index) for index in range(len(data))}

print(result_json)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [35]:
len(result_json)

192885

In [39]:
# Convert int64 to regular integers
def convert_int64(obj):
    if isinstance(obj, np.int64):
        return int(obj)
    raise TypeError

json_result = {int(key): value for key, value in result_json.items()}

with open('facebook-pair-mining.json', 'w') as json_file:
    json.dump(json_result, json_file, indent=4, default = convert_int64)

In [ ]:
data[6]['cleaned_text']

In [ ]:
data[57006]['cleaned_text']

In [40]:
with open('facebook-pair-mining.json') as json_file:
    dat = json.load(json_file)

In [42]:
dat['50']

{'positive': [43644, 78217, 15770, 91392, 172623, 90454, 168396],
 'negative': [159503,
  145138,
  63754,
  181566,
  10828,
  167890,
  127536,
  29695,
  106669,
  39862,
  107284,
  29126,
  15123,
  131744,
  42373,
  2819,
  118357,
  69860,
  101545,
  20483,
  30543,
  9985]}

In [48]:
texts[168396]

'Saifudin abdullah'

In [44]:
texts[50]

'Mohd Fuad Abdullah Sani kah'